In [1]:
email_conversation = """From: 테디 (teddy@teddynote.com)
To: 이은채 대리님 (eunchae@teddyinternational.me)
Subject: RAG 솔루션 시연 관련 미팅 제안

안녕하세요, 이은채 대리님,

저는 테디노트의 테디입니다. 최근 귀사에서 AI를 활용한 혁신적인 솔루션을 모색 중이라는 소식을 들었습니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 노하우를 가진 기업으로, 귀사의 요구에 맞는 최적의 솔루션을 제공할 수 있다고 자부합니다.

저희 테디노트의 RAG 솔루션은 귀사의 데이터 활용을 극대화하고, 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보입니다. 이 솔루션은 특히 다양한 산업에서의 성공적인 적용 사례를 통해 그 효과를 입증하였습니다.

귀사와의 협력 가능성을 논의하고, 저희 RAG 솔루션의 구체적인 기능과 적용 방안을 시연하기 위해 미팅을 제안드립니다. 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 만나 뵐 수 있을까요?

미팅 시간을 조율하기 어려우시다면, 편하신 다른 일정을 알려주시면 감사하겠습니다. 이은채 대리님과의 소중한 만남을 통해 상호 발전적인 논의가 이루어지길 기대합니다.

감사합니다.

테디
테디노트 AI 솔루션팀"""

In [2]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# .env 파일 로드
load_dotenv()

# langSmith에 로깅 할 프로젝트 명을 입력
logging.langsmith("03-OutputParser-project")

LangSmith 추적을 시작합니다.
[프로젝트명]
03-OutputParser-project


In [ ]:
from pydantic import BaseModel, Field


# 이메일 본문으로부터 주요 엔티티 추출
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    company: str = Field(description="메일을 보낸 사람의 회사 정보")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [4]:
### Reminder

## LCEL 구조
# chain = prompt | llm | output_parser

In [ ]:
# llm 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [ ]:
# output_parser 정의
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [7]:
# prompt 정의
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """ 
        You are a helpful assistant. Please answer the following questions in KOREAN.

        #QUESTION:
        Please extract the key points from the following email.

        #EMAIL CONVERSATION:
        {email_conversation}

        #FORMAT:
        {format}
    """
)
prompt = prompt.partial(format=output_parser.get_format_instructions())

print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "company": {"description": "메일을 보낸 사람의 회사 정보", "title": "Company", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "company", "email", 

In [8]:
# 체인 생성
chain = prompt | llm | output_parser

In [9]:
# 체인 실행
answer = chain.invoke({"email_conversation": email_conversation})

print(answer)

person='테디' company='테디노트' email='teddy@teddynote.com' subject='RAG 솔루션 시연 관련 미팅 제안' summary='테디노트의 테디가 이은채 대리님에게 AI 및 RAG 솔루션 시연을 위한 미팅을 제안합니다. 테디노트는 AI 솔루션 분야에서 다양한 경험을 가지고 있으며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보이는 RAG 솔루션을 제공합니다. 미팅은 다음 주 목요일(7월 18일) 오전 10시에 제안되었으며, 다른 일정이 필요하면 조율 가능합니다.' date='7월 18일 오전 10시'


In [10]:
print(answer.date)

7월 18일 오전 10시


In [11]:
print(answer.summary)

테디노트의 테디가 이은채 대리님에게 AI 및 RAG 솔루션 시연을 위한 미팅을 제안합니다. 테디노트는 AI 솔루션 분야에서 다양한 경험을 가지고 있으며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보이는 RAG 솔루션을 제공합니다. 미팅은 다음 주 목요일(7월 18일) 오전 10시에 제안되었으며, 다른 일정이 필요하면 조율 가능합니다.


## 검색: SERP API

참고: https://serpapi.com/integrations/python

In [ ]:
from langchain_community.utilities import SerpAPIWrapper
import os

params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}

search = SerpAPIWrapper(
    params=params, serpapi_api_key=os.environ.get("SERPAPI_API_KEY")
)

In [13]:
os.environ.get("SERPAPI_API_KEY")

'09ab7d2f227c9bb3a29ab969d210e1cd030bb09133a956773efd37f331cb5a91'

In [14]:
query = f"{answer.person} {answer.company} {answer.email}"
query

'테디 테디노트 teddy@teddynote.com'

In [15]:
search_result = search.run(query)

In [16]:
search_result

'[\'테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 (Windows). LocalModels. GGUF · HuggingFace gguf 파일을 Ollama 로딩 · TeddyNote.\', \'테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...\', \'데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \\u200d♂️ ...more ...more fastcampus.co.kr/data_online_teddyand 2 ...\']'

In [17]:
type(search_result)

str

In [18]:
search_result = eval(search_result)
search_result

['테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 (Windows). LocalModels. GGUF · HuggingFace gguf 파일을 Ollama 로딩 · TeddyNote.',
 '테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...',
 '데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d♂️ ...more ...more fastcampus.co.kr/data_online_teddyand 2 ...']

In [19]:
type(search_result)

list

In [22]:
# 검색 결과
search_result_string = "\n".join(search_result)
type(search_result_string)

str

In [21]:
answer

EmailSummary(person='테디', company='테디노트', email='teddy@teddynote.com', subject='RAG 솔루션 시연 관련 미팅 제안', summary='테디노트의 테디가 이은채 대리님에게 AI 및 RAG 솔루션 시연을 위한 미팅을 제안합니다. 테디노트는 AI 솔루션 분야에서 다양한 경험을 가지고 있으며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보이는 RAG 솔루션을 제공합니다. 미팅은 다음 주 목요일(7월 18일) 오전 10시에 제안되었으며, 다른 일정이 필요하면 조율 가능합니다.', date='7월 18일 오전 10시')

In [ ]:
from langchain_core.prompts import PromptTemplate

report_prompt = PromptTemplate.from_template(
    """ 
        당신은 이메일의 주요 정보를 바탕으로 요약 정리해 주는 전문가 입니다.
        당신의 임무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
        주어진 정보를 기반으로 양식(format)에 맞추어 요약을 작성해 주세요.

        #Information:
        - Sender: {sender}
        - Additional Information about sender: {additional_information}
        - Company: {company}
        - Email: {email}
        - Subject: {subject}
        - Summary: {summary}
        - Date: {date}

        #Format(in markdown format):
        보낸 사람 :
        - (보낸 사람의 이름, 회사 정보)

        이메일 주소:
        - (보낸 사람의 이메일 주소)

        보낸 사람과 관련하여 검색된 추가 정보:
        - (검색된 추가 정보)

        주요 내용:
        - (이메일 제목, 요약)

        일정 :
        - (미팅 날짜 및 시간)

        #Answer:
    """
)

In [24]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
report_chain = report_prompt | llm | StrOutputParser()

In [ ]:
report_response = report_chain.invoke(
    {
        "sender": answer.person,
        "company": answer.company,
        "email": answer.email,
        "subject": answer.subject,
        "summary": answer.summary,
        "date": answer.date,
        "additional_information": search_result_string,
    }
)

In [28]:
print(report_response)

보낸 사람 :
- 테디, 테디노트

이메일 주소:
- teddy@teddynote.com

보낸 사람과 관련하여 검색된 추가 정보:
- 테디는 테디노트와 패스트캠퍼스의 "RAG 비법노트"와 관련된 환경 설정을 다루며, 데이터와 인공지능을 좋아하는 개발자입니다. 그는 데이터 분석, 머신러닝, 딥러닝, LLM에 대한 내용을 다루며, 연구보다는 개발에 더 많은 관심을 가지고 있습니다. 테디노트는 AI 솔루션 분야에서 다양한 경험을 보유하고 있으며, 데이터 활용을 극대화하고 비즈니스 의사결정을 지원하는 RAG 솔루션을 제공합니다.

주요 내용:
- 이메일 제목: RAG 솔루션 시연 관련 미팅 제안
- 요약: 테디노트의 테디가 이은채 대리님에게 AI 및 RAG 솔루션 시연을 위한 미팅을 제안합니다. 미팅은 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보이는 RAG 솔루션을 소개하기 위한 것입니다. 미팅은 다음 주 목요일(7월 18일) 오전 10시에 제안되었으며, 다른 일정이 필요하면 조율 가능합니다.

일정 :
- 7월 18일 오전 10시
